In [1]:
from nltk.tokenize import word_tokenize
import nltk
from pprint import pprint  # pretty-printer
import gensim
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"
from gensim.models.doc2vec import Doc2Vec

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/madis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/madis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/madis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# import data from data folder
import pandas as pd
data = pd.read_csv('data/train.csv')

list_of_text = data['text'].to_list()
list_of_targets = data['target'].to_list()

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim import corpora

en_stop = set(stopwords.words('english'))

def get_lemma2(word):
    from nltk.stem.wordnet import WordNetLemmatizer
    """
    This function takes a word and returns its rootword
    """
    return WordNetLemmatizer().lemmatize(word)

def nltk_preprocessing(text):
    """
    This function takes a text and returns a list of tokens
    - lowercase
    - remove short words
    - remove stopwords
    - gets root word (lemma)

    """
    tokens = word_tokenize(text.lower())
    # tokens = [token for token in tokens if len(token) > 4]      # remove short words
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma2(token) for token in tokens]
    return tokens

# apply some tf-idf


In [4]:
# apply tf-idf
from gensim import corpora, models

# Create the Dictionary and Corpus
mydict = corpora.Dictionary([nltk_preprocessing(line) for line in list_of_text])
corpus = [mydict.doc2bow(nltk_preprocessing(line)) for line in list_of_text]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]  

In [5]:
# corpus tfidf to scipy sparse matrix
from gensim.matutils import corpus2csc
import scipy.sparse
corpus_tfidf_sparse = scipy.sparse.csc_matrix(corpus2csc(corpus_tfidf))



# Doc to Vec

In [33]:
from gensim. models.doc2vec import TaggedDocument as TaggedDocument
from gensim.models import Doc2Vec

list_of_preprocessed_text = [nltk_preprocessing(line) for line in list_of_text]

def taggedDoc_func(list_of_preprocessed_text, list_of_targets):
    taggedDoc = []
    for text, target in zip(list_of_preprocessed_text, list_of_targets):
        taggedDoc.append(TaggedDocument(text, str(target)))
    return taggedDoc
    
taggedDoc = taggedDoc_func(list_of_preprocessed_text, list_of_targets)

In [34]:
import gensim
size=400
#instantiate our DM and DBOW models
model_dm = gensim.models.Doc2Vec(min_count=1, window=10, vector_size=size, sample=1e-3, negative=5, workers=3)
model_dbow = gensim.models.Doc2Vec(min_count=1, window=10, vector_size=size, sample=1e-3, negative=5, dm=0, workers=3)

In [35]:
import numpy as np
#build vocab over all reviews -- can only be once generated

model_dm.build_vocab(taggedDoc)
model_dbow.build_vocab(taggedDoc) 

In [37]:
# TRAIN MODES
#We pass through the data set multiple times, shuffling the training reviews each time to improve accuracy.
for epoch in range(10):
    perm = np.random.permutation(taggedDoc.shape[0])
    model_dm.train(taggedDoc[perm])
    model_dbow.train(taggedDoc[perm])

AttributeError: 'list' object has no attribute 'shape'

In [46]:
all_tagged_doc = np.asanyarray(taggedDoc, dtype=object)
for epoch in range(10):
    perm = np.random.permutation(all_tagged_doc.shape[0])
        model_dm.train(all_tagged_doc[perm], total_examples=model_dm.corpus_count, epochs=10)
        model_dbow.train(all_tagged_doc[perm], total_examples=model_dbow.corpus_count,  epochs=10)

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/madis/sandbox/cohere-takehome/venv/lib/python3.8/site-packages/gensim/models/base_any2vec.py", line 251, in _job_producer
    data_length = self._raw_word_count([data])
  File "/home/madis/sandbox/cohere-takehome/venv/lib/python3.8/site-packages/gensim/models/doc2vec.py", line 873, in _raw_word_count
    return sum(len(sentence.words) for sentence in job)
  File "/home/madis/sandbox/cohere-takehome/venv/lib/python3.8/site-packages/gensim/models/doc2vec.py", line 873, in <genexpr>
    return sum(len(sentence.words) for sentence in job)
AttributeError: 'numpy.ndarray' object has no attribute 'words'


KeyboardInterrupt: 

In [47]:
# save models to disk
model_dm.save('model_dm.model')
model_dbow.save('model_dbow.model')